<h2> 0. IMPORT REQUIRED LIBRARIES </h2>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging

<h2> 1. SET INITIAL REQUIREMENTS </h2>

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
query = 'monitor'
base_url = 'https://listado.mercadolibre.com.ve/'
products = []
page_num = 1
full_url = base_url + query

<h2>2. LOOP PAGES AND EXTRACT ALL PRODUCTS </h2>

In [ ]:
while True:
    url = f'{full_url}_Desde_{page_num}'
    logging.info(f"Scraping URL: {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching URL {url}: {e}")
        break

    product_elements = soup.find_all('li', class_='ui-search-layout__item')

    if not product_elements:
        logging.info("No more product elements found. Stopping scraping.")
        break

    for element in product_elements:
        title_element = element.find('h2', class_='ui-search-item__title')
        title = title_element.text if title_element else None

        price_element = element.find('span', class_='price-tag-fraction')
        price = float(price_element.text.replace('.', '')) if price_element else None

        link_element = element.find('a', class_='ui-search-link')
        link = link_element['href'] if link_element else None

        if price and price >= 50:
            product = {'title': title, 'price': price, 'link': link}
            products.append(product)

    page_num += 50
    time.sleep(3)  # Respectful delay to avoid overloading the server

df = pd.DataFrame(products)

<h2>3. SORT AND FILTER EXTRACTED DATA </h2>

In [ ]:
df = df.sort_values(by='price', ascending=True)
df = df[(df['price'] >= 100) & (df['price'] < 1000)].reset_index(drop=True)
df

<h2>4. EXPORT FINAL DATAFRAME </h2>

In [ ]:
try:
    df.to_excel('products.xlsx', index=False)
    logging.info(f"DataFrame exported to products.xlsx")
except Exception as e:
    logging.error(f"Error exporting to Excel: {e}")